In [1]:
import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import shutil
from glob import glob
import pandas_profiling as pp
import cv2
from google.cloud import storage
from kaggle_datasets import KaggleDatasets
from random import seed, randint, random, choice
from PIL import Image
import tensorflow_addons as tfa

In [2]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [3]:
IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600)
im_size = IMSIZES[7]

load_dir = "/kaggle/input/plant-pathology-2021-fgvc8/"
df = pd.read_csv(load_dir + 'train.csv')

strategy = auto_select_accelerator()
BATCH_SIZE = 32

test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

n_labels = 5

Running on 1 replicas


In [4]:
strategy = auto_select_accelerator()
BATCH_SIZE = 32

test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

Running on 1 replicas


In [5]:
from keras.preprocessing.image import ImageDataGenerator

testgen = ImageDataGenerator(
            preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=(0.8, 1.2),
            rescale=1/255.0)

test_set = testgen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_dir,
                                    x_col="image",
                                    y_col=None,
                                    batch_size=BATCH_SIZE,
                                    seed=42,
                                    shuffle=False,
                                    class_mode=None,
                                    target_size=(im_size,im_size))

Found 3 validated image filenames.


In [6]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, GlobalAveragePooling2D, ReLU, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D, GlobalMaxPooling2D
import tensorflow_addons

with strategy.scope():
    base = tf.keras.applications.EfficientNetB7(weights=None,
                                                include_top=False, 
                                                input_shape=(im_size, im_size, 3),
                                                drop_connect_rate=0.4)

    model_69 = Sequential()
    model_69.add(base)
    model_69.add(GlobalAveragePooling2D())
    model_69.add(Dense(n_labels, activation = 'sigmoid'))
model_69.load_weights('../input/3modelsgive08/besteffb7_2.h5')

In [7]:
with strategy.scope():
    base = tf.keras.applications.EfficientNetB7(weights=None,
                                                include_top=False, 
                                                input_shape=(im_size, im_size, 3),
                                                drop_connect_rate=0.4)
    model_71 = Sequential()
    model_71.add(base)
    model_71.add(GlobalAveragePooling2D())
    model_71.add(Dense(n_labels, activation = 'sigmoid'))
model_71.load_weights('../input/3modelsgive08/besteffb7.h5')

In [8]:
with strategy.scope():
    base = tf.keras.applications.EfficientNetB7(weights=None,
                                                include_top=False, 
                                                input_shape=(im_size, im_size, 3),
                                                drop_connect_rate=0.4)
    model_72 = Sequential()
    model_72.add(base)
    model_72.add(GlobalMaxPooling2D())
    model_72.add(Dense(n_labels, activation = 'sigmoid'))
model_72.load_weights('../input/3modelsgive08/bestplantmodel_1model.h5')

In [9]:
TTA = 3 
preds = []

for i in range(TTA):
    test_set.reset()
#     preds.append(model_69.predict(test_set))
#     preds.append(model_71.predict(test_set))
    preds.append(model_72.predict(test_set))
    
pred = np.mean(np.array(preds), axis=0)

In [10]:
name = {0: 'complex',
        1: 'scab',
        2: 'frog_eye_leaf_spot',
        3: 'rust',
        4: 'powdery_mildew',
        6: 'healthy'}
one_hot = {'healthy':            [0, 0, 0, 0, 0],
           'complex':            [1, 0, 0, 0, 0],
           'scab':               [0, 1, 0, 0, 0],
           'frog_eye_leaf_spot': [0, 0, 1, 0, 0],
           'rust':               [0, 0, 0, 1, 0],
           'powdery_mildew':     [0, 0, 0, 0, 1],
           'scab frog_eye_leaf_spot': [0, 1, 1, 0, 0],
           'rust frog_eye_leaf_spot': [0, 0, 1, 1, 0],
           'frog_eye_leaf_spot complex': [1, 0, 1, 0, 0],
           'scab frog_eye_leaf_spot complex': [1, 1, 1, 0, 0],
           'powdery_mildew complex': [1, 0, 0, 0, 1],
           'rust complex': [1, 0, 0, 1, 0]}
threshold = {0: 0.25,
             1: 0.6,
             2: 0.7,
             3: 0.5,
             4: 0.5}

def get_key(val):
    for key, value in name.items():
        if val == value:
            return key
 
    return "key doesn't exist"

pred_string = []
for line in pred:
    s = ''
    p=np.array(line.tolist())
    il = one_hot.values()
    i = np.argmin([np.linalg.norm(l-p) for l in il])
    s = [ key  for key in one_hot if one_hot[key]==list(il)[i]][0]

    pred_string.append(s)
    
test_df['labels'] = pred_string
test_df.to_csv('submission.csv', index=False)
test_df.head()

,image,labels
0,ad8770db05586b59.jpg,complex
1,c7b03e718489f3ca.jpg,frog_eye_leaf_spot
2,85f8cb619c66b863.jpg,healthy


In [11]:

pred

array([[6.6689682e-01, 6.6740505e-02, 1.4975788e-01, 5.0694346e-03,
        1.5962094e-02],
       [2.0183849e-01, 6.7533753e-03, 8.2991379e-01, 1.2634993e-03,
        1.1414066e-04],
       [2.0842890e-01, 2.4351032e-01, 6.1664473e-02, 9.5889680e-02,
        2.5063545e-02]], dtype=float32)

In [12]:
# !cp ../input/effmodel1-2/besteffb7.h5 .
# !cp ../input/effnetb7-2/besteffb7_2.h5 .
# !cp ../input/plant-1-model-2/bestplantmodel_1model.h5 .